In [1]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import asyncio
import pandas as pd

In [5]:
url = "https://seller.unicro.co.kr/buyer/orderList/1"
JSESSIONID="A25610A4650F6A28641CCAA2C744171C.tc_seller2"
headers={"Cookie": f"JSESSIONID={JSESSIONID}"}

In [6]:
def parse_row(row):
    divs = row.find_all("div")
    spans = divs[2].find_all("span", recursive=False)
    return {
        "url": row.find("a")['href'],
        "ordered_at": datetime.fromisoformat(divs[0].text),
        "title": divs[1].text.replace("\r","").replace("\n",""),
        "amount": spans[0].text.strip().split("원")[0].replace(",", ""),
        "status": spans[2].text.strip()
    }

async def fetch_detail_and_append(item):
    res = requests.get(f"""https://seller.unicro.co.kr{item['url']}""", headers=headers)
    soup = BeautifulSoup(res.text)
    form = soup.find("div", class_="orderform")
    divs = form.find_all("div", recursive=False)
    print(f"""Fetched detail for {item["title"]}!""")
    return {
        **item,
        **{x.find("th").text.strip(): x.find_all("td", recursive=False)[-1].text.strip() for x in divs[1].find_all("tr")},
        **{x.find("th").text.strip(): x.find_all("td", recursive=False)[-1].text.strip() for x in divs[2].find_all("tr")},
        **{x.find("th").text.strip(): x.find_all("td", recursive=False)[-1].text.strip() for x in divs[3].find_all("tr")},
    }

async def load():
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text)
    rows = [x.find_all("td")[-1] for x in soup.find("table").find_all("tr") if len(x.find_all('th')) == 0]
    data = [parse_row(x) for x in rows]
    fetched_data = await asyncio.gather(*[fetch_detail_and_append(x) for x in data])
    return fetched_data

In [7]:
data = await load()

Fetched detail for *균일 가격 인하* 빈폴스포츠 D085 블랙 보아 구스 다운 코트 43.9만원 >> 5만원/블랙/95/1/합배!
Fetched detail for ♡빈폴 가을신상 최대80%♡[빈폴 레이디스][SOPHIE DOWN] 그레이 오버사이즈 소피 다운 점퍼 979.000 >> 22만!!/그레이/90/1!
Fetched detail for *균일 가격 인하* 빈폴스포츠 C01 여성 리브 퀼팅 구스 다운 점퍼 (블랙/화이트) 45.9만원 >> 5만원/블랙/90/1!
Fetched detail for 시스템 신상 ★ 모캐시미어 벨셋 248 후드코트 ★ 로즈베이지/로즈베이지/55/1/합배!
Fetched detail for 더캐시 넥배색 원피스/아이보리/프리사이즈/1!
Fetched detail for 타임 핑크색 벨티드 패딩/핑크색/프리사이즈/1!
Fetched detail for *균일 가격 인하* 빈폴스포츠 E06 여성 마카롱 푸퍼 미드 다운 (네이비/ 레몬) 42.9만원 >> 5만원 85 90 95 100/레몬/85/1!
Fetched detail for 백플랩 오트밀 266 니트원피스 / 오트밀 / 90 TM1JAKOP266W!
Fetched detail for *균일 가격 인하* 빈폴스포츠 E01 여성 투웨이 점퍼&조끼 다운 베스트 (블랙/ 화이트) 42.9만원 >> 5만원/블랙/105/1!
Fetched detail for *균일 가격 인하* 빈폴스포츠 C01 여성 리브 퀼팅 구스 다운 점퍼 (블랙/화이트) 45.9만원 >> 5만원/블랙/95/1!
Fetched detail for ♡빈폴스포츠 균일가SALE♡네이비 남성 DUAL 구스 다운 399.000 >> 5만!!/네이비/95,100/2/합배!
Fetched detail for ♡빈폴스포츠 균일가SALE♡남성 롱 구스 다운 아이보리 439.000 >> 5만!!/아이보리/95,110/2/합배!
Fetched detail for *균일 가격 인하* 빈폴스포츠 D085 블랙 보아 구

Fetched detail for 시스템 신상 ☆ 울100 프런트 바디,슬리브에 크로쉐 디테일을 더해준 블라우스 ★ 카키브라운/브라운/66/1!
Fetched detail for ♡ 1년차 70%초특가세일♡!!벌륜소매 진주 포인트 니트 가디건 앞뒤로 두가지 연출가능 ☆113700☆/민트/프리/1!
Fetched detail for Sjsj 신상 기장감 딱 이뻐요~곧 가볍게 걸치기 좋으실 핸드메이드 더블코트 2컬러~/베이지/44/2/합배!
Fetched detail for 타임 80퍼 정가115만 럭셔리한 시그니처 세련된 핸드메이드 롱 모혼방 더블자켓/wg/55/1!
Fetched detail for Sjsj일년차비품 몰인기~!! 뒷태에반한 가볍고 바스락 벨티드 후드 트렌치 코트 55한점/페일베이지/55/1/합배!
Fetched detail for 씨씨콜렉트 ☆ 특가 ■멋스런 헤링본 벨티드 캐시미어 루즈핏 코트/카멜/55/1!
Fetched detail for 시스템 신상 추천드려요~~!깔별로 겟 하시는 매장인기템~보들촉감 심플 기본 터틀넥 니트 3컬러~/오프화이트/프리/1/합배!
Fetched detail for 시스템 신상★ 루즈하고 스타일리쉬한 오버핏 220 더블코트/베이지/55/1!
Fetched detail for 시스템 신상★ 데일로 좋은~ 프런트 언발라인 모100 오프화이트컬러 355니트탑/오프화이트/프리/1!
Fetched detail for 시스템 신상 놓치면 후회하실 분위기 멋스런 두고두고 입어질 핸드메이드 롱코트/베이지/55/1/ 합배!
Fetched detail for 시스템 신상 놓치면 후회하실 분위기 멋스런 두고두고 입어질 핸드메이드 롱코트/베이지/55/1!
Fetched detail for 씨씨콜렉트 ☆ 반짝 특가 ■폭스퍼 벨티드 베이지 구스패딩(소량만요~~)/베이지/55/1!
Fetched detail for 타임 80% 특가 시그니처]인기라인~ 따루 활용도 좋은 우아한 랩 스타일 스플릿원피스 55한점/쿨그레이/55/1!
Fetche

In [49]:
pending = [x for x in data if x["status"] not in ["미입금", "배송완료", "환불완료", "중복요청서"]]

In [50]:
df = pd.DataFrame(pending)

In [51]:
df.to_clipboard()

In [10]:
df = pd.DataFrame(data)
df.to_clipboard()

In [28]:
detail

{'주문서명': 'Madam L 주문서',
 '주문번호': '211108103751',
 '구매대행 이용약관': '동의',
 '카페 닉네임': '일꾼',
 '셀러명': 'Madam L',
 '주문일시': '2021-11-08 14:56:39',
 '현재상태': '주문(2021-11-08 14:56:39)',
 '입금자 성함': '안장현',
 '입금금액': '130,350원',
 '상품명': '시스템 신상★ 상콤옐로컬러! 케이블짜임 010 니트탑/옐로우/프리/1/합배',
 '받는사람 성함': '안장현',
 '받는사람 전화번호': '01079260500',
 '우편번호': '06109',
 '받는사람 주소': '서울 강남구 봉은사로37길 7-9 (논현동) 지하1층29호 사서함 BCJ8E',
 '배송 시 요청사항': '부재시 문앞에 놔주세요.',
 '이전 Madam L주문상품 확인': '없음',
 '바로배송or킵요청': '바로배송',
 '킵상품': '',
 '결제수단': '현금',
 '현금영수증 신청': '사업자지출증빙',
 '현금영수증 정보': '2128701637',
 '환불계좌정보': '기업은행\n  04711489504015'}